In [1]:
# !pip install geopandas 
# !pip install matplotlib
# %pip install pygeos
# %pip install rtree

In [46]:
# All the import 
import numpy as np
import pandas as pd

In [47]:
# read files
data = pd.read_csv("../data/owid-covid-data.csv")

In [ ]:
# print(data.columns)
data.tail()

In [ ]:
# Basic information about all the columns of the data
data.info()

In [ ]:
df = data[data['continent'] == 'Asia'] 
df['year'] = pd.DatetimeIndex(df['date']).year
df['month'] = pd.DatetimeIndex(df['date']).month
df['population_million'] = df["population"]/1000000

In [ ]:
cases_df = df[["iso_code", "location","population","population_million","population_density","year", "month","new_cases", "new_cases_per_million", "new_cases_smoothed_per_million" ]]
cases_df

In [ ]:
deaths_df = df[["iso_code", "location","population","population_million","population_density","year", "month","new_deaths", "new_deaths_per_million", "new_deaths_smoothed_per_million" ]]
deaths_df

In [52]:
unique_lpd_val = cases_df[["location", "population", "population_million","population_density"]]
unique_lpd_val.shape

(53116, 4)

In [53]:
unique_lpd =  unique_lpd_val.drop_duplicates().reset_index()
#unique_lpd[unique_lpd["population_density"].isna()]

In [ ]:
dff = df.groupby(by=["location"])['new_cases','new_cases_per_million', 'new_deaths', 'new_deaths_per_million' ].sum().reset_index()
dff

In [ ]:
#merge_pop_cases_deaths_sum = dff.merge(uniqe_lpd_val,  left_on='location', right_on='location').drop_duplicates()
merge_pop_cases_deaths_sum = pd.merge(dff, unique_lpd, on='location', how='inner').drop("index", axis =1)
merge_pop_cases_deaths_sum

In [56]:
#to do a fair comparision for number of deaths per population I am thinking of doing per million analysis
#merge_pop_cases_deaths_sum["new_cases_million"] = (merge_pop_cases_deaths_sum["new_cases"] /merge_pop_cases_deaths_sum["population"]) * 1000000

In [125]:
sankey_data = merge_pop_cases_deaths_sum[["location", "new_cases_per_million", "new_deaths_per_million"]].sort_values(by="new_cases_per_million", ascending=False).reset_index().drop("index", axis=1)

In [127]:
sankey_data = sankey_data.loc[(sankey_data["new_cases_per_million"] > 0) & (sankey_data["new_deaths_per_million"] > 0)]


In [ ]:
sankey_data

In [30]:
# data_for_top_six.to_csv (r'E:\Documents\GitHub\Data-Visualization\data\scatter_plot_data.csv', index = False, header=True)

In [129]:
def make_json(data):
#     print(data)

    def get_nodeval(val, dList):
        for index in range(len(dList)):
            for value in dList[index].values():
                if val == value:
                    return(dList[index]['node'])
        return
    
    def adduniquenode(graph, name):
        if not any(d['name'] == name for d in graph['nodes']):
            graph['nodes'].append({ 'node': len(graph['nodes']), 'name': name })
            
    def adduniquelink(data, graph, node, valueof ):
        if valueof == "new_cases_per_million":
            for index, row in data.iterrows():
                graph['links'].append({"source" : get_nodeval(valueof, graph['nodes']),
                              "target" : get_nodeval(row['location'], graph['nodes']),
                              "value" : row[valueof]                       
                             })
        else:
            for index, row in data.iterrows():
                graph['links'].append({"source" : get_nodeval(row['location'], graph['nodes'] ),
                              "target" : get_nodeval(valueof, graph['nodes']),
                              "value" : row[valueof]                       
                             })
            
#         print(graph['nodes'])   

    
    graph = {"nodes" : [], "links" : []}
#     print(graph['nodes'].push())
    keys = data.columns[1:]
    print(data.columns)
    adduniquenode(graph, keys[0])
    adduniquenode(graph, keys[1])
    for i in range(data.shape[0]):
#         graph['nodes'].push()
        adduniquenode(graph, data.loc[i].location)
#     print(graph['nodes'])

    adduniquelink(data, graph, 0, 'new_cases_per_million')
    adduniquelink(data, graph, 1, 'new_deaths_per_million')
#     print(graph['links'])
    
    return graph

In [91]:
# To display complete values of the coulmns 
# with pd.option_context('display.max_colwidth', None, 'display.max_rows', None): display(polydata)

In [132]:
json_string = make_json(sankey_data)

Index(['location', 'new_cases_per_million', 'new_deaths_per_million'], dtype='object')


In [ ]:
json_string

In [ ]:
''''dList = [{'node':0, 'a': 3}, {'node':1, 'a': 4}, {'node':2, 'a': 5}]
val = 3
for index in range(len(dList)):
    for value in dList[index].values():
        if val == value:
            print(dList[index]['node'])
#         print(dList[index][key])'''

In [101]:
#sankey_data.to_csv("..\data\sankey_data.csv",index=False,header=True)

In [134]:
import json

In [135]:
# Directly from dictionary
with open('../data/sankey_data.json', 'w') as outfile:
    json.dump(json_string, outfile)